In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import copy

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]

def equals(a, b, tol = 1E-3):
    if abs(a-b) <= tol:
        return True
    else:
        return False
    
def equals_array(arr_1, arr_2, tol = 1E-2):
    if np.linalg.norm(arr_1 - arr_2) <= tol:
        return True
    else:
        return False
    

In [3]:
def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

In [4]:
# arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
# np.linalg.norm(arr_1)

# Download Groups, Routes to Edges Data:

In [5]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [6]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [7]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [8]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [9]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

edge_to_od_dict = {}
num_edges = int(np.max(od_to_edges_array)) + 1
# print("num_edges:", num_edges)

for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
# od_to_edges_array
# edge_to_od_dict

In [10]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

[[ 47.77809012  27.56428276 115.15744798 134.75871573 287.28108034]
 [  7.64672375   4.41157139  18.43056493  21.56768237  45.97837742]
 [ 88.76462936  51.21036309 213.94551691 250.3617751  533.7257842 ]
 [ 19.95836002  11.51443847  48.10476517  56.29281031 120.00603652]
 [ 75.66495318  43.65285761 182.37193845 213.41397052 454.95978261]
 [ 18.9161492   10.913163    45.59276988  53.35324135 113.73940996]
 [119.4231777   68.89798713 287.84047959 336.83460377 718.07013258]
 [ 15.23267296  22.71991899  85.716058    74.8724603   59.63978734]
 [  3.42495327   5.10840487  19.27261838  16.83451606  13.40956279]
 [ 12.98400593  19.36597495  73.06254185  63.81969017  50.83568424]
 [  3.2461412    4.84170212  18.26642165  15.95560927  12.70946808]
 [ 20.49356972  30.56668025 115.31974823 100.73110538  80.23753567]
 [  7.61874311   9.23040031  31.06102962  37.36114412  61.2429735 ]
 [ 28.88291825  34.99276635 117.75343596 141.63738759 232.1742275 ]
 [  7.2210114    8.74853304  29.43950801  35.410

In [11]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [12]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [13]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': np.float64(861.9885),
  'Latency (at bend)': np.float64(1.326448252),
  'Slope (after bend)': np.float64(0.000782666)},
 'East Palo Alto': {'Flow (at bend)': np.float64(1001.517857),
  'Latency (at bend)': np.float64(2.213126553),
  'Slope (after bend)': np.float64(0.000584484)},
 'Redwood City': {'Flow (at bend)': np.float64(881.1846667),
  'Latency (at bend)': np.float64(4.892192375),
  'Slope (after bend)': np.float64(0.001563724)},
 'Belmont': {'Flow (at bend)': np.float64(1278.948125),
  'Latency (at bend)': np.float64(1.199911179),
  'Slope (after bend)': np.float64(0.001994138)},
 'San Mateo': {'Flow (at bend)': np.float64(1034.092826),
  'Latency (at bend)': np.float64(5.541006284),
  'Slope (after bend)': np.float64(0.002147262)},
 'Burlingame': {'Flow (at bend)': np.float64(845.15),
  'Latency (at bend)': np.float64(1.503111345),
  'Slope (after bend)': np.float64(0.000306601)},
 'Millbrae': {'Flow (at bend)': np.float64(853.1818182),
  'Laten

In [14]:
# num_edges = 7
num_gp_lanes = 3

num_el = 3
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]
    

In [15]:
## Set lambdas:

lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

# directory_inits = '../data/opt_values___2_el_groups/'
directory_inits = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
df_inits = pd.read_csv(directory_inits + filename_segment + '___tau_B_stats_CBCP.csv')

print("filename_segment:", filename_segment)
print()

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

argmin_tau = np.zeros((num_edges, T))
argmin_B = 0

for e in range(num_edges):
    for t in range(T):
        argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
        argmin_B = inits_B_arr_as_object

print("argmin_tau:\n", argmin_tau)
print()
print("argmin_B:\n", argmin_B)


filename_segment: 1_1_1

argmin_tau:
 [[0.92 0.28 0.46 0.48 0.94]
 [0.95 0.32 0.8  0.58 0.7 ]
 [0.51 2.23 2.08 1.35 2.19]
 [1.27 0.86 1.72 1.38 2.47]
 [2.13 2.29 2.25 0.58 1.31]
 [0.53 0.44 0.69 0.5  0.38]
 [0.79 0.26 0.47 0.36 0.75]]

argmin_B:
 7.02


In [16]:
# argmin_tau = np.array([[0.0, 0.3194, 0.3194, 0.0, 0.0], \
#                        [0.2498, 0.2498, 0.2498, 0.0, 0.0], \
#                        [0.0, 0.0, 0.0, 0.9995, 0.9995], \
#                        [0.0, 1.0281, 1.0281, 1.0281, 0.0], \
#                        [1.6043, 0.0, 0.0, 1.6043, 0.0], \
#                        [0.0, 0.1922, 0.1922, 0.1922, 0.0], \
#                        [0.0, 0.0, 0.0, 0.2178, 0.2178]])

# argmin_B = 10.6925

# Compute CBCP and DBCP comparison statistics

In [17]:
# demand_array
# edge_to_od_dict
# od_to_edges_array

# demand_array[:, 0:3]
# demand_array[:, 3:5]
# demand_array

# np.sum(demand_array[:, 0:3], axis=1).shape

In [18]:
demand_array_el = np.sum(demand_array[:, 0:3], axis=1)
demand_array_in = np.sum(demand_array[:, 3:5], axis=1)

demand_array_el_across_edges = np.zeros(num_edges)
demand_array_in_across_edges = np.zeros(num_edges)
demand_array_across_edges = np.zeros(num_edges)

for e in range(num_edges):
    demand_array_el_across_edges[e] = np.sum([demand_array_el[od] for od in edge_to_od_dict[e]])
    demand_array_in_across_edges[e] = np.sum([demand_array_in[od] for od in edge_to_od_dict[e]])
    demand_array_across_edges[e] = demand_array_el_across_edges[e] + demand_array_in_across_edges[e]

# demand_array_el_across_edges[e]

In [19]:
demand_array_el_across_edges

array([1507.7602297 , 1766.88182121, 2298.02265605, 1990.28188162,
       2635.7926891 , 2188.02527639, 2341.16466505])

In [20]:
demand_array_across_edges

array([4848.10363249, 5174.23084722, 6793.71793018, 6047.6382991 ,
       7944.86026387, 6645.07072326, 7113.41109776])

In [21]:
# Read data from csv and store into numpy files

directory_CBCP = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
directory_DBCP = '../data/opt_DBCP_values___' + str(num_el) + '_el_groups/'

tau_CBCP_dict = {}
tau_time_averaged_CBCP_dict = {}
B_CBCP_dict = {}
percent_express_lane_use_CBCP_dict = {}
avg_travel_time_CBCP_dict = {}
total_costs_CBCP_dict = {}

tau_DBCP_dict = {}
tau_time_averaged_DBCP_dict = {}
alpha_DBCP_dict = {}
percent_express_lane_use_DBCP_dict = {}
avg_travel_time_DBCP_dict = {}
total_costs_DBCP_dict = {}


# lambdas_array = np.array([[1.0, 1.0, 1.0], \
#                           [1.0, 5.0, 1.0], \
#                           [1.0, 10.0, 1.0], \
#                           [5.0, 5.0, 1.0], \
#                           [5.0, 10.0, 1.0], \
#                           [10.0, 10.0, 1.0], \
#                           [1.0, 1.0, 0.0], \
#                           [1.0, 5.0, 0.0], \
#                           [5.0, 10.0, 0.0]])

lambdas_array = np.array([[1.0, 1.0, 1.0], \
                          [1.0, 5.0, 1.0], \
                          [1.0, 10.0, 1.0], \
                          [5.0, 5.0, 1.0], \
                          [5.0, 10.0, 1.0], \
                          [10.0, 10.0, 1.0], \
                          [1.0, 1.0, 0.0], \
                          [1.0, 5.0, 0.0], \
                          [5.0, 10.0, 0.0], \
                          [5.0, 1.0, 1.0], \
                          [10.0, 5.0, 1.0], \
                          [10.0, 1.0, 1.0], \
                          [20.0, 1.0, 1.0], \
                          [5.0, 1.0, 0.0], \
                          [10.0, 5.0, 0.0], \
                          [10.0, 1.0, 0.0], \
                          [20.0, 1.0, 0.0], \
                          [5.0, 0.0, 1.0], \
                          [10.0, 0.0, 1.0], \
                          [20.0, 0.0, 1.0], \
                         ])

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    
    filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))
    
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    lambdas = (lambda_E, lambda_R, lambda_I)
    filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

    # Inputting CBCP data:

    df_CBCP = pd.read_csv(directory_CBCP + filename_segment + '___tau_B_stats_CBCP.csv')
    tau_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 1:6].astype(float)
    tau_time_averaged_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 6].astype(float)
    B_CBCP_dict[lambdas] = df_CBCP.to_numpy()[0, 7]
    percent_express_lane_use_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 8:11].astype(float)
    avg_travel_time_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 11:13].astype(float)
    total_costs_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 13:].astype(float)

    # Inputting DBCP data:

    df_DBCP = pd.read_csv(directory_DBCP + filename_segment + '___tau_alpha_stats_DBCP.csv')
    tau_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 1:6].astype(float)
    tau_time_averaged_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 6].astype(float)
    alpha_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 7:12].astype(float)
    percent_express_lane_use_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 12:15].astype(float)
    avg_travel_time_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 15:17].astype(float)
    total_costs_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 17:].astype(float)
    


In [22]:
 total_costs_DBCP_dict[(5.0, 10.0, 1.0)]

array([[  2366.6 ,  39977.47,   1214.34,  39667.05],
       [  4128.18,  57334.01,   1072.04,  67254.54],
       [ 13745.55, 193095.54,   4328.19, 218541.34],
       [  3526.97,  51783.96,   2000.23,  49416.54],
       [ 19278.06, 272238.63,   5136.14, 317267.49],
       [  3806.31,  53322.85,   1828.27,  54071.7 ],
       [  6190.79,  85572.51,   1065.26, 105873.82]])

In [23]:
# Process data into average or total costs, and storing into dict_avg_stats:

dict_avg_stats = {}

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    lambdas = (lambda_E, lambda_R, lambda_I)

    # Processing CBCP data:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'CBCP')
    
    dict_avg_stats[dict_index] = {}
    
    dict_avg_stats[dict_index]['percent of overall'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
            / np.sum(demand_array_across_edges)
    dict_avg_stats[dict_index]['percent of eligible'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 1] * demand_array_el_across_edges) \
            / np.sum(demand_array_el_across_edges)
    dict_avg_stats[dict_index]['percent of ineligible'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 2] * demand_array_in_across_edges) \
            / np.sum(demand_array_in_across_edges)
    
    dict_avg_stats[dict_index]['average travel time, express'] \
        = np.sum(avg_travel_time_CBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['average travel time, general purpose'] \
        = np.sum(avg_travel_time_CBCP_dict[lambdas][:, 1])
    
    dict_avg_stats[dict_index]['total travel cost, eligible'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['total travel cost, ineligible'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 1])
    dict_avg_stats[dict_index]['total toll revenue'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 2])
    dict_avg_stats[dict_index]['total societal cost'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 3])

#     # Processing DBCP data:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'DBCP')
    
    dict_avg_stats[dict_index] = {}
    
    dict_avg_stats[dict_index]['percent of overall'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
            / np.sum(demand_array_across_edges)
    dict_avg_stats[dict_index]['percent of eligible'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 1] * demand_array_el_across_edges) \
            / np.sum(demand_array_el_across_edges)
    dict_avg_stats[dict_index]['percent of ineligible'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 1] * demand_array_in_across_edges) \
            / np.sum(demand_array_in_across_edges)
    
    dict_avg_stats[dict_index]['average travel time, express'] \
        = np.sum(avg_travel_time_DBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['average travel time, general purpose'] \
        = np.sum(avg_travel_time_DBCP_dict[lambdas][:, 1])
    
    dict_avg_stats[dict_index]['total travel cost, eligible'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['total travel cost, ineligible'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 1])
    dict_avg_stats[dict_index]['total toll revenue'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 2])
    dict_avg_stats[dict_index]['total societal cost'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 3])
    

In [24]:
# lambdas = (1.0, 1.0, 1.0)

# percent_express_lane_use_CBCP_dict[lambdas][:, 0]

# np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
#             / np.sum(demand_array_across_edges)

# dict_avg_stats[(1.0, 1.0, 1.0, 'CBCP')]

In [25]:
avg_stats_array = np.zeros((lambdas_array.shape[0] * 2, 13))

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    
    # Storing CBCP avg stats as array:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'CBCP')
    
    avg_stats_array[2 * lambdas_index, 0] = lambda_E
    avg_stats_array[2 * lambdas_index, 1] = lambda_R
    avg_stats_array[2 * lambdas_index, 2] = lambda_I
    
    # Here, "0" indicates CBCP
    avg_stats_array[2 * lambdas_index, 3] = 0
    
    avg_stats_array[2 * lambdas_index, 4] = dict_avg_stats[dict_index]['percent of overall']
    avg_stats_array[2 * lambdas_index, 5] = dict_avg_stats[dict_index]['percent of eligible']
    avg_stats_array[2 * lambdas_index, 6] = dict_avg_stats[dict_index]['percent of ineligible']
    
    avg_stats_array[2 * lambdas_index, 7] = dict_avg_stats[dict_index]['average travel time, express']
    avg_stats_array[2 * lambdas_index, 8] = dict_avg_stats[dict_index]['average travel time, general purpose']

    avg_stats_array[2 * lambdas_index, 9] = dict_avg_stats[dict_index]['total travel cost, eligible']
    avg_stats_array[2 * lambdas_index, 10] = dict_avg_stats[dict_index]['total travel cost, ineligible']
    avg_stats_array[2 * lambdas_index, 11] = dict_avg_stats[dict_index]['total toll revenue']
    avg_stats_array[2 * lambdas_index, 12] = dict_avg_stats[dict_index]['total societal cost']
    
    # Storing DBCP avg stats as array:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'DBCP')
    
    avg_stats_array[2 * lambdas_index + 1, 0] = lambda_E
    avg_stats_array[2 * lambdas_index + 1, 1] = lambda_R
    avg_stats_array[2 * lambdas_index + 1, 2] = lambda_I
    
    # Here, "1" indicates DBCP
    avg_stats_array[2 * lambdas_index + 1, 3] = 1
    
    avg_stats_array[2 * lambdas_index + 1, 4] = dict_avg_stats[dict_index]['percent of overall']
    avg_stats_array[2 * lambdas_index + 1, 5] = dict_avg_stats[dict_index]['percent of eligible']
    avg_stats_array[2 * lambdas_index + 1, 6] = dict_avg_stats[dict_index]['percent of ineligible']
    
    avg_stats_array[2 * lambdas_index + 1, 7] = dict_avg_stats[dict_index]['average travel time, express']
    avg_stats_array[2 * lambdas_index + 1, 8] = dict_avg_stats[dict_index]['average travel time, general purpose']

    avg_stats_array[2 * lambdas_index + 1, 9] = dict_avg_stats[dict_index]['total travel cost, eligible']
    avg_stats_array[2 * lambdas_index + 1, 10] = dict_avg_stats[dict_index]['total travel cost, ineligible']
    avg_stats_array[2 * lambdas_index + 1, 11] = dict_avg_stats[dict_index]['total toll revenue']
    avg_stats_array[2 * lambdas_index + 1, 12] = dict_avg_stats[dict_index]['total societal cost']


In [26]:
avg_stats_array.shape

(40, 13)

In [27]:
column_names = []
column_names += ["lambda_E", "lambda_R", "lambda_I"]
column_names += ["CBCP (=0) or DBCP (=1)"]
column_names += ["% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

df_avg_stats_to_save = pd.DataFrame(avg_stats_array, index=None, columns=column_names)

df_avg_stats_to_save

,lambda_E,lambda_R,lambda_I,CBCP (=0) or DBCP (=1),% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
0,1.0,1.0,1.0,0.0,17.050196,33.006506,9.175478,21.43,24.95,51159.77,756941.12,13986.99,794113.87
1,1.0,1.0,1.0,1.0,22.838640,26.384389,26.406051,23.02,24.15,51927.25,733605.55,10622.55,774910.23
2,1.0,5.0,1.0,0.0,16.567084,35.495381,7.223786,21.37,25.13,51231.10,762057.05,10464.71,760964.68
3,1.0,5.0,1.0,1.0,19.999887,24.790138,24.616483,22.03,24.46,52579.51,743756.60,17890.24,706884.89
4,1.0,10.0,1.0,0.0,16.682506,25.014403,12.565155,21.12,25.10,51923.48,760354.98,22431.21,587966.35
5,1.0,10.0,1.0,1.0,17.785132,13.863931,13.791203,21.42,24.92,53470.78,754842.57,29145.22,516861.18
6,5.0,5.0,1.0,0.0,17.906063,35.289622,9.326668,21.48,25.07,51506.33,760379.60,14316.90,946326.71
7,5.0,5.0,1.0,1.0,19.194933,26.157331,26.184580,21.84,24.59,52694.71,746896.29,20425.92,908240.21
8,5.0,10.0,1.0,0.0,14.269659,33.851220,4.607741,21.28,25.52,51813.72,772144.53,9448.54,936727.94
9,5.0,10.0,1.0,1.0,18.401244,32.517994,32.407623,21.70,24.78,53042.46,753324.97,16644.47,852092.48


In [28]:
directory_to_save = "../data/stats_compare/"
# filename = "opt_CBCP_params___" + random_string + '.csv'

filename = 'avg_stats.csv'

df_avg_stats_to_save.to_csv(directory_to_save + filename)

In [36]:
# Stats array are: DBCP minus CBCP

stats_compare_percentage_array = np.zeros((lambdas_array.shape[0], 11))

# for lambdas_index in range(lambdas_array.shape[0]):
for lambdas_index in [0]:
    print()
    print("lambdas_index:", lambdas_index)
    print("avg_stats_array[2 * lambdas_index, 4:12]:\n", avg_stats_array[2 * lambdas_index, 4:12])
    
    stats_compare_percentage_array[lambdas_index, 0:3] = lambdas_array[lambdas_index]
    
    stats_compare_percentage_array[lambdas_index, 3:] = \
        100 * (avg_stats_array[2 * lambdas_index + 1, 4:12] - avg_stats_array[2 * lambdas_index, 4:12]) \
            / avg_stats_array[2 * lambdas_index, 4:12]




lambdas_index: 0
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.70501962e+01 3.30065056e+01 9.17547798e+00 2.14300000e+01
 2.49500000e+01 5.11597700e+04 7.56941120e+05 1.39869900e+04]

lambdas_index: 1
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.65670840e+01 3.54953813e+01 7.22378600e+00 2.13700000e+01
 2.51300000e+01 5.12311000e+04 7.62057050e+05 1.04647100e+04]

lambdas_index: 2
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.66825058e+01 2.50144032e+01 1.25651555e+01 2.11200000e+01
 2.51000000e+01 5.19234800e+04 7.60354980e+05 2.24312100e+04]

lambdas_index: 3
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.79060635e+01 3.52896221e+01 9.32666791e+00 2.14800000e+01
 2.50700000e+01 5.15063300e+04 7.60379600e+05 1.43169000e+04]

lambdas_index: 4
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.42696587e+01 3.38512199e+01 4.60774149e+00 2.12800000e+01
 2.55200000e+01 5.18137200e+04 7.72144530e+05 9.44854000e+03]

lambdas_index: 5
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.57948216e+01 3.5

/var/folders/wc/5q980jhx25l5t8_jtg5hp94m0000gn/T/ipykernel_4867/3560470676.py:13: RuntimeWarning: divide by zero encountered in divide
  100 * (avg_stats_array[2 * lambdas_index + 1, 4:12] - avg_stats_array[2 * lambdas_index, 4:12]) \


In [30]:
# stats_compare_percentage_array

In [ ]:
# arr_1 = np.array([1, 2, 3])
# arr_2 = np.array([1.1, 2.5, 3.333])

# (arr_2 - arr_1) / arr_1 * 100

In [ ]:
column_names = []
column_names += ["lambda_E", "lambda_R", "lambda_I"]
column_names += ["% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue"]

df_stats_compare_to_save = pd.DataFrame(stats_compare_percentage_array, index=None, columns=column_names)

df_stats_compare_to_save

In [ ]:
directory_to_save = "../data/stats_compare/"
# filename = "opt_CBCP_params___" + random_string + '.csv'

filename = 'compare_stats.csv'

df_stats_compare_to_save.to_csv(directory_to_save + filename)

## <font color='red'>STOP. End of Code to run.</font> 

# End of code to run.

In [ ]:
# np.sum(np.array([1, 2, 3]) * np.array([5, 3, 1]))


In [ ]:
# avg_travel_time_CBCP_dict[(1.0, 1.0, 1.0)]

In [ ]:
# tau_CBCP_dict = {}
# tau_time_averaged_CBCP_dict = {}
# B_CBCP_dict = {}
# percent_express_lane_use_CBCP_dict = {}
# avg_travel_time_CBCP_dict = {}
# total_costs_CBCP_dict = {}

# tau_DBCP_dict = {}
# tau_time_averaged_DBCP_dict = {}
# alpha_DBCP_dict = {}
# percent_express_lane_use_DBCP_dict = {}
# avg_travel_time_DBCP_dict = {}
# total_costs_DBCP_dict = {}


# directory_CBCP = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
# directory_DBCP = '../data/opt_DBCP_values___' + str(num_el) + '_el_groups/'

# lambdas_index = 4
# lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
# lambdas = (lambda_E, lambda_R, lambda_I)
# filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

# # Inputting CBCP data:

# df_CBCP = pd.read_csv(directory_CBCP + filename_segment + '___tau_B_stats_CBCP.csv')
# tau_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 1:6]
# tau_time_averaged_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 6]
# B_CBCP_dict[lambdas] = df_CBCP.to_numpy()[0, 7]
# percent_express_lane_use_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 8:11]
# avg_travel_time_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 11:12]
# total_costs_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 12:]

# # Inputting DBCP data:

# df_DBCP = pd.read_csv(directory_DBCP + filename_segment + '___tau_alpha_stats_DBCP.csv')
# tau_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 1:6]
# tau_time_averaged_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 6]
# alpha_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 7:12]
# percent_express_lane_use_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 12:15]
# avg_travel_time_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 15:16]
# total_costs_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 16:]



## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
